In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests

import sys
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install folium
from geopy.geocoders import Nominatim
import folium

from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Getting the data table from the wiki page

In [2]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_data, 'html5lib')
table = soup.find('table')

# Cleaning up the table

In [3]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


# Grouping the neighborhoods of each location

In [4]:
joindf = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: '%s' % ', '.join(x))
joindf = joindf.reset_index(drop=False)
joindf.rename(columns={'Neighborhood':'Neighborhood_joined'}, inplace=True)
joindf.head()

,PostalCode,Neighborhood_joined
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


# Merging the grouped table with borough

In [5]:
final = pd.merge(df, joindf, on='PostalCode')
final.drop(['Neighborhood'], axis=1, inplace=True)
final.drop_duplicates(inplace=True)
final.rename(columns={'Neighborhood_joined':'Neighborhood'}, inplace=True)
final.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
final.shape

(103, 3)

# Getting geospatial data

In [7]:
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
merging = pd.merge(geo, final, on='PostalCode')

toronto = merging[merging['Borough'].str.contains("Toronto")]
toronto.reset_index(drop=True, inplace=True)
toronto.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M4E,43.676357,-79.293031,East Toronto,The Beaches
1,M4J,43.685347,-79.338106,East York/East Toronto,The Danforth East
2,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
3,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
4,M4M,43.659526,-79.340923,East Toronto,Studio District


In [9]:
tmap = folium.Map(location=[43.68, -79.34], zoom_start=8)

for la, lg, bo, nei in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(nei, bo)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [la, lg],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3132cc',
        parse_html=False).add_to(tmap)  
    
tmap